# WHIS Model Promotion Pipeline

Automated pipeline for promoting experimental models to production with validation and deployment.

## Objectives
- Validate experimental models against production criteria
- Automated model testing and performance benchmarking
- Safe deployment with rollback capabilities
- Model monitoring and drift detection setup

## Pipeline Stages
1. **Model Validation** - Performance, stability, security checks
2. **Integration Testing** - API compatibility, load testing
3. **Staging Deployment** - Shadow mode testing
4. **Production Promotion** - Blue-green deployment
5. **Monitoring Setup** - Performance tracking, alerts


In [ ]:
import pandas as pd
import numpy as np
import json
import joblib
from pathlib import Path
from datetime import datetime, timedelta
import shutil
import subprocess
import time
import requests
import warnings
warnings.filterwarnings('ignore')

# ML imports
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_curve
from sklearn.model_selection import cross_val_score

# WHIS imports
import sys
sys.path.append('../models')
from isolation_forest_anomaly import WhisAnomalyDetector

print("🚀 WHIS Model Promotion Pipeline")
print("=" * 35)

# Configuration
CONFIG = {
    'model_artifacts_dir': Path('../models/artifacts'),
    'staging_dir': Path('../models/staging'),
    'production_dir': Path('../models/production'),
    'validation_data_dir': Path('../feature_store/tables'),
    'api_endpoint': 'http://localhost:8000',
    'performance_thresholds': {
        'min_auc': 0.70,
        'min_precision': 0.60,
        'min_recall': 0.50,
        'max_inference_time_ms': 100,
        'max_model_size_mb': 50
    },
    'deployment_config': {
        'shadow_mode_duration_hours': 24,
        'rollback_threshold_error_rate': 0.05,
        'monitoring_check_interval_minutes': 5
    }
}

# Ensure directories exist
for dir_path in [CONFIG['staging_dir'], CONFIG['production_dir']]:
    dir_path.mkdir(parents=True, exist_ok=True)

print(f"📁 Pipeline directories configured")
print(f"  • Artifacts: {CONFIG['model_artifacts_dir']}")
print(f"  • Staging: {CONFIG['staging_dir']}")
print(f"  • Production: {CONFIG['production_dir']}")

## 1. Model Discovery & Selection

In [ ]:
def discover_candidate_models():
    """Find trained models available for promotion"""
    
    artifacts_dir = CONFIG['model_artifacts_dir']
    candidates = []
    
    if artifacts_dir.exists():
        for model_dir in artifacts_dir.iterdir():
            if model_dir.is_dir() and (model_dir / 'metadata.json').exists():
                # Load metadata
                with open(model_dir / 'metadata.json', 'r') as f:
                    metadata = json.load(f)
                
                # Check if required files exist
                required_files = ['models.joblib', 'scalers.joblib', 'encoders.joblib']
                if all((model_dir / f).exists() for f in required_files):
                    candidates.append({
                        'path': model_dir,
                        'name': metadata.get('model_name', model_dir.name),
                        'created_at': metadata.get('created_at'),
                        'version': metadata.get('version', '1.0'),
                        'model_types': metadata.get('model_types', []),
                        'metadata': metadata
                    })
    
    # Sort by creation date (newest first)
    candidates.sort(key=lambda x: x['created_at'], reverse=True)
    
    return candidates

# Discover available models
candidate_models = discover_candidate_models()

print(f"🔍 Discovered {len(candidate_models)} candidate models:")
if candidate_models:
    for i, model in enumerate(candidate_models, 1):
        print(f"  {i}. {model['name']} (v{model['version']})")
        print(f"     Created: {model['created_at']}")
        print(f"     Types: {', '.join(model['model_types'])}")
        print(f"     Path: {model['path']}")
        print()
else:
    print("  No trained models found!")
    print("  Run anomaly detection training first.")

# Select most recent model for promotion
if candidate_models:
    selected_model = candidate_models[0]
    print(f"🎯 Selected for promotion: {selected_model['name']}")
else:
    selected_model = None
    print("❌ No models available for promotion")

## 2. Model Validation

In [ ]:
def validate_model_performance(model_path, validation_data_dir):
    """Comprehensive model validation against production criteria"""
    
    print(f"🔬 Validating model: {model_path}")
    print("=" * 30)
    
    validation_results = {
        'timestamp': datetime.now().isoformat(),
        'model_path': str(model_path),
        'passed': False,
        'tests': {},
        'performance_metrics': {},
        'issues': []
    }
    
    try:
        # Load the model
        detector = WhisAnomalyDetector.load_model(model_path)
        
        # Load validation data
        auth_df = pd.read_parquet(validation_data_dir / "auth_events.parquet")
        process_df = pd.read_parquet(validation_data_dir / "process_events.parquet")
        admin_df = pd.read_parquet(validation_data_dir / "admin_events.parquet")
        
        datasets = {
            'auth_events': auth_df,
            'process_events': process_df,
            'admin_events': admin_df
        }
        
        print("\n📊 Performance Validation:")
        
        all_tests_passed = True
        
        for table_type, df in datasets.items():
            if table_type not in detector.models:
                continue
                
            print(f"\n🧪 Testing {table_type}...")
            
            # Prepare features
            X = detector.prepare_features(df, table_type)
            y = df['is_suspicious'] if 'is_suspicious' in df.columns else None
            
            if y is None:
                print(f"   ⚠️  No labels available for {table_type}")
                continue
            
            # Performance testing
            start_time = time.time()
            anomaly_scores = detector.predict_anomaly_score(df, table_type)
            inference_time = (time.time() - start_time) * 1000  # ms
            
            # Calculate metrics
            auc = roc_auc_score(y, anomaly_scores) if len(np.unique(y)) > 1 else 0.5
            
            # Binary predictions for precision/recall
            threshold = np.percentile(anomaly_scores, 85)  # Top 15% as anomalies
            y_pred = (anomaly_scores > threshold).astype(int)
            
            report = classification_report(y, y_pred, output_dict=True, zero_division=0)
            precision = report.get('1', {}).get('precision', 0.0)
            recall = report.get('1', {}).get('recall', 0.0)
            
            # Store metrics
            validation_results['performance_metrics'][table_type] = {
                'auc': auc,
                'precision': precision,
                'recall': recall,
                'inference_time_ms': inference_time,
                'samples_tested': len(df)
            }
            
            # Check thresholds
            thresholds = CONFIG['performance_thresholds']
            
            tests = {
                'auc_test': auc >= thresholds['min_auc'],
                'precision_test': precision >= thresholds['min_precision'],
                'recall_test': recall >= thresholds['min_recall'],
                'inference_time_test': inference_time <= thresholds['max_inference_time_ms']
            }
            
            validation_results['tests'][table_type] = tests
            
            # Report results
            print(f"   AUC: {auc:.3f} {'✅' if tests['auc_test'] else '❌'} (min: {thresholds['min_auc']})")
            print(f"   Precision: {precision:.3f} {'✅' if tests['precision_test'] else '❌'} (min: {thresholds['min_precision']})")
            print(f"   Recall: {recall:.3f} {'✅' if tests['recall_test'] else '❌'} (min: {thresholds['min_recall']})")
            print(f"   Inference: {inference_time:.1f}ms {'✅' if tests['inference_time_test'] else '❌'} (max: {thresholds['max_inference_time_ms']}ms)")
            
            # Check if all tests passed for this table
            if not all(tests.values()):
                all_tests_passed = False
                failed_tests = [test for test, passed in tests.items() if not passed]
                validation_results['issues'].append(f"{table_type}: Failed tests - {', '.join(failed_tests)}")
        
        # Model size check
        model_size_mb = sum(f.stat().st_size for f in model_path.rglob('*') if f.is_file()) / (1024 * 1024)
        size_test_passed = model_size_mb <= CONFIG['performance_thresholds']['max_model_size_mb']
        
        print(f"\n📦 Model Size: {model_size_mb:.1f}MB {'✅' if size_test_passed else '❌'} (max: {CONFIG['performance_thresholds']['max_model_size_mb']}MB)")
        
        if not size_test_passed:
            all_tests_passed = False
            validation_results['issues'].append(f"Model size too large: {model_size_mb:.1f}MB")
        
        validation_results['passed'] = all_tests_passed
        
        if all_tests_passed:
            print("\n✅ All validation tests PASSED!")
        else:
            print("\n❌ Validation FAILED with issues:")
            for issue in validation_results['issues']:
                print(f"   • {issue}")
        
    except Exception as e:
        validation_results['issues'].append(f"Validation error: {str(e)}")
        print(f"\n💥 Validation failed with error: {e}")
    
    return validation_results

# Run validation if we have a selected model
if selected_model:
    validation_results = validate_model_performance(
        selected_model['path'], 
        CONFIG['validation_data_dir']
    )
    
    # Save validation report
    report_path = CONFIG['staging_dir'] / f"validation_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    with open(report_path, 'w') as f:
        json.dump(validation_results, f, indent=2)
    
    print(f"\n💾 Validation report saved: {report_path}")
else:
    validation_results = None
    print("⚠️  Skipping validation - no model selected")

## 3. Staging Deployment

In [ ]:
def deploy_to_staging(model_path, validation_results):
    """Deploy validated model to staging environment"""
    
    print("🏗️  Deploying to staging environment...")
    print("=" * 35)
    
    if not validation_results or not validation_results.get('passed', False):
        print("❌ Cannot deploy - model failed validation")
        return None
    
    try:
        # Create staging deployment
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        staging_path = CONFIG['staging_dir'] / f"whis_anomaly_detector_staging_{timestamp}"
        
        # Copy model artifacts
        shutil.copytree(model_path, staging_path)
        
        # Create staging metadata
        staging_metadata = {
            'deployment_type': 'staging',
            'deployed_at': datetime.now().isoformat(),
            'source_model': str(model_path),
            'validation_passed': True,
            'validation_report': validation_results,
            'staging_path': str(staging_path),
            'status': 'active',
            'shadow_mode': True,
            'performance_monitoring': {
                'start_time': datetime.now().isoformat(),
                'end_time': (datetime.now() + timedelta(hours=CONFIG['deployment_config']['shadow_mode_duration_hours'])).isoformat(),
                'metrics_collected': [],
                'issues_detected': []
            }
        }
        
        # Save staging metadata
        with open(staging_path / 'staging_metadata.json', 'w') as f:
            json.dump(staging_metadata, f, indent=2)
        
        # Create symlink to latest staging
        latest_staging = CONFIG['staging_dir'] / 'latest'
        if latest_staging.exists():
            latest_staging.unlink()
        latest_staging.symlink_to(staging_path.name)
        
        print(f"✅ Staging deployment successful!")
        print(f"   Path: {staging_path}")
        print(f"   Shadow mode duration: {CONFIG['deployment_config']['shadow_mode_duration_hours']} hours")
        
        return {
            'staging_path': staging_path,
            'metadata': staging_metadata,
            'deployment_time': datetime.now().isoformat()
        }
        
    except Exception as e:
        print(f"💥 Staging deployment failed: {e}")
        return None

# Deploy to staging if validation passed
if validation_results and validation_results.get('passed', False):
    staging_deployment = deploy_to_staging(selected_model['path'], validation_results)
else:
    staging_deployment = None
    if validation_results:
        print("⚠️  Skipping staging deployment - validation failed")
    else:
        print("⚠️  Skipping staging deployment - no validation results")

## 4. Integration Testing

In [ ]:
def run_integration_tests(staging_deployment):
    """Run integration tests against staging deployment"""
    
    print("🧪 Running integration tests...")
    print("=" * 30)
    
    if not staging_deployment:
        print("❌ No staging deployment available")
        return None
    
    integration_results = {
        'timestamp': datetime.now().isoformat(),
        'staging_path': str(staging_deployment['staging_path']),
        'tests': {},
        'passed': False,
        'issues': []
    }
    
    try:
        # Test 1: Model Loading
        print("\n🔧 Test 1: Model Loading")
        try:
            detector = WhisAnomalyDetector.load_model(staging_deployment['staging_path'])
            integration_results['tests']['model_loading'] = {'passed': True, 'message': 'Model loaded successfully'}
            print("   ✅ Model loads correctly")
        except Exception as e:
            integration_results['tests']['model_loading'] = {'passed': False, 'message': f'Model loading failed: {e}'}
            integration_results['issues'].append(f'Model loading: {e}')
            print(f"   ❌ Model loading failed: {e}")
            return integration_results
        
        # Test 2: Prediction API
        print("\n🔧 Test 2: Prediction Functionality")
        try:
            # Create sample data for each model type
            test_data = {
                'auth_events': pd.DataFrame({
                    'hour_of_day': [3, 10],
                    'is_weekend': [True, False],
                    'is_off_hours': [True, False],
                    'fail_count_1h': [5, 0],
                    'success_after_fail_15m': [True, False],
                    'is_admin': [True, False],
                    'asset_class': ['server', 'workstation']
                }),
                'process_events': pd.DataFrame({
                    'hour_of_day': [2, 14],
                    'cmd_len': [150, 50],
                    'cmd_entropy': [4.5, 2.0],
                    'has_encoded': [True, False],
                    'signed_parent': [False, True],
                    'rare_parent_child_7d': [True, False]
                }),
                'admin_events': pd.DataFrame({
                    'off_hours': [True, False],
                    'recent_4625s_actor_1h': [3, 0],
                    'method': ['registry', 'group']
                })
            }
            
            prediction_tests_passed = True
            
            for table_type, sample_df in test_data.items():
                if table_type in detector.models:
                    scores = detector.predict_anomaly_score(sample_df, table_type)
                    
                    # Validate scores
                    if len(scores) == len(sample_df) and all(0 <= score <= 1 for score in scores):
                        print(f"   ✅ {table_type}: Predictions valid (scores: {scores.round(3)})")
                    else:
                        prediction_tests_passed = False
                        integration_results['issues'].append(f'{table_type}: Invalid prediction scores')
                        print(f"   ❌ {table_type}: Invalid prediction scores")
            
            integration_results['tests']['prediction_api'] = {
                'passed': prediction_tests_passed,
                'message': 'All prediction tests passed' if prediction_tests_passed else 'Some prediction tests failed'
            }
            
        except Exception as e:
            integration_results['tests']['prediction_api'] = {'passed': False, 'message': f'Prediction test failed: {e}'}
            integration_results['issues'].append(f'Prediction API: {e}')
            print(f"   ❌ Prediction test failed: {e}")
        
        # Test 3: Performance Stress Test
        print("\n🔧 Test 3: Performance Stress Test")
        try:
            # Generate larger test dataset
            stress_data = pd.DataFrame({
                'hour_of_day': np.random.randint(0, 24, 1000),
                'is_weekend': np.random.choice([True, False], 1000),
                'is_off_hours': np.random.choice([True, False], 1000),
                'fail_count_1h': np.random.randint(0, 10, 1000),
                'success_after_fail_15m': np.random.choice([True, False], 1000),
                'is_admin': np.random.choice([True, False], 1000),
                'asset_class': np.random.choice(['server', 'workstation', 'mobile'], 1000)
            })
            
            # Time the prediction
            start_time = time.time()
            stress_scores = detector.predict_anomaly_score(stress_data, 'auth_events')
            stress_time = (time.time() - start_time) * 1000  # ms
            
            throughput = len(stress_data) / (stress_time / 1000)  # samples/second
            
            stress_test_passed = stress_time <= CONFIG['performance_thresholds']['max_inference_time_ms'] * 10  # 10x buffer for stress
            
            integration_results['tests']['stress_test'] = {
                'passed': stress_test_passed,
                'message': f'Processed {len(stress_data)} samples in {stress_time:.1f}ms ({throughput:.1f} samples/sec)',
                'throughput_samples_per_sec': throughput
            }
            
            if stress_test_passed:
                print(f"   ✅ Stress test passed: {throughput:.1f} samples/sec")
            else:
                integration_results['issues'].append(f'Stress test failed: {stress_time:.1f}ms too slow')
                print(f"   ❌ Stress test failed: {stress_time:.1f}ms too slow")
            
        except Exception as e:
            integration_results['tests']['stress_test'] = {'passed': False, 'message': f'Stress test failed: {e}'}
            integration_results['issues'].append(f'Stress test: {e}')
            print(f"   ❌ Stress test failed: {e}")
        
        # Overall result
        all_tests_passed = all(test.get('passed', False) for test in integration_results['tests'].values())
        integration_results['passed'] = all_tests_passed
        
        if all_tests_passed:
            print("\n✅ All integration tests PASSED!")
        else:
            print("\n❌ Some integration tests FAILED:")
            for issue in integration_results['issues']:
                print(f"   • {issue}")
        
    except Exception as e:
        integration_results['issues'].append(f'Integration test error: {e}')
        print(f"\n💥 Integration testing failed: {e}")
    
    return integration_results

# Run integration tests
if staging_deployment:
    integration_results = run_integration_tests(staging_deployment)
    
    # Save integration test report
    if integration_results:
        report_path = CONFIG['staging_dir'] / f"integration_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        with open(report_path, 'w') as f:
            json.dump(integration_results, f, indent=2)
        print(f"\n💾 Integration test report saved: {report_path}")
else:
    integration_results = None
    print("⚠️  Skipping integration tests - no staging deployment")

## 5. Production Promotion Decision

In [ ]:
def make_promotion_decision(validation_results, integration_results):
    """Automated decision for production promotion"""
    
    print("🎯 Making production promotion decision...")
    print("=" * 40)
    
    decision = {
        'timestamp': datetime.now().isoformat(),
        'promote_to_production': False,
        'reasons': [],
        'blockers': [],
        'recommendations': []
    }
    
    # Check validation results
    if not validation_results or not validation_results.get('passed', False):
        decision['blockers'].append('Model failed performance validation')
        if validation_results and 'issues' in validation_results:
            decision['blockers'].extend(validation_results['issues'])
    else:
        decision['reasons'].append('Model passed all validation tests')
        
        # Highlight strong performance
        for table_type, metrics in validation_results.get('performance_metrics', {}).items():
            auc = metrics.get('auc', 0)
            if auc >= 0.85:
                decision['reasons'].append(f'{table_type} shows excellent performance (AUC: {auc:.3f})')
    
    # Check integration results
    if not integration_results or not integration_results.get('passed', False):
        decision['blockers'].append('Model failed integration tests')
        if integration_results and 'issues' in integration_results:
            decision['blockers'].extend(integration_results['issues'])
    else:
        decision['reasons'].append('Model passed all integration tests')
        
        # Check throughput
        stress_test = integration_results.get('tests', {}).get('stress_test', {})
        if stress_test.get('passed', False):
            throughput = stress_test.get('throughput_samples_per_sec', 0)
            decision['reasons'].append(f'Good performance throughput: {throughput:.1f} samples/sec')
    
    # Make final decision
    if not decision['blockers']:
        decision['promote_to_production'] = True
        decision['recommendations'] = [
            'Deploy with blue-green strategy for safe rollback',
            'Monitor performance metrics for first 24 hours',
            'Set up automated drift detection alerts',
            'Plan regular model retraining schedule'
        ]
    else:
        decision['recommendations'] = [
            'Fix identified issues before promotion',
            'Re-run validation and integration tests',
            'Consider model architecture improvements',
            'Review training data quality'
        ]
    
    # Report decision
    if decision['promote_to_production']:
        print("\n🎉 PROMOTION APPROVED!")
        print("\n✅ Reasons for approval:")
        for reason in decision['reasons']:
            print(f"   • {reason}")
    else:
        print("\n🚫 PROMOTION BLOCKED!")
        print("\n❌ Blocking issues:")
        for blocker in decision['blockers']:
            print(f"   • {blocker}")
    
    print("\n💡 Recommendations:")
    for rec in decision['recommendations']:
        print(f"   • {rec}")
    
    return decision

# Make promotion decision
if validation_results or integration_results:
    promotion_decision = make_promotion_decision(validation_results, integration_results)
    
    # Save decision report
    decision_path = CONFIG['staging_dir'] / f"promotion_decision_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    with open(decision_path, 'w') as f:
        json.dump(promotion_decision, f, indent=2)
    
    print(f"\n💾 Promotion decision saved: {decision_path}")
else:
    promotion_decision = None
    print("⚠️  Cannot make promotion decision - no test results available")

## 6. Production Deployment (If Approved)

In [ ]:
def deploy_to_production(staging_deployment, promotion_decision):
    """Deploy approved model to production with blue-green strategy"""
    
    if not promotion_decision or not promotion_decision.get('promote_to_production', False):
        print("⚠️  Skipping production deployment - not approved")
        return None
    
    print("🚀 Deploying to PRODUCTION environment...")
    print("=" * 40)
    
    try:
        # Backup current production model (if exists)
        current_production = CONFIG['production_dir'] / 'current'
        if current_production.exists():
            backup_path = CONFIG['production_dir'] / f"backup_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
            shutil.move(current_production, backup_path)
            print(f"📦 Backed up current production to: {backup_path}")
        
        # Deploy new model
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        production_path = CONFIG['production_dir'] / f"whis_anomaly_detector_prod_{timestamp}"
        
        # Copy from staging
        shutil.copytree(staging_deployment['staging_path'], production_path)
        
        # Create production metadata
        production_metadata = {
            'deployment_type': 'production',
            'deployed_at': datetime.now().isoformat(),
            'source_staging': str(staging_deployment['staging_path']),
            'validation_passed': True,
            'integration_passed': True,
            'promotion_approved': True,
            'version': timestamp,
            'status': 'active',
            'blue_green_deployment': {
                'deployment_strategy': 'blue_green',
                'cutover_time': datetime.now().isoformat(),
                'rollback_available': True,
                'monitoring_active': True
            },
            'monitoring': {
                'drift_detection': True,
                'performance_tracking': True,
                'error_rate_monitoring': True,
                'alert_thresholds': {
                    'max_error_rate': 0.05,
                    'min_throughput_samples_per_sec': 100,
                    'max_inference_time_ms': 50
                }
            }
        }
        
        # Save production metadata
        with open(production_path / 'production_metadata.json', 'w') as f:
            json.dump(production_metadata, f, indent=2)
        
        # Create symlink to current production
        current_production.symlink_to(production_path.name)
        
        # Create deployment manifest
        manifest = {
            'deployment_id': f"whis-anomaly-{timestamp}",
            'version': timestamp,
            'model_path': str(production_path),
            'deployment_time': datetime.now().isoformat(),
            'status': 'deployed',
            'health_check_url': f"{CONFIG['api_endpoint']}/health",
            'rollback_command': f"ln -sfn backup_{timestamp} {current_production}"
        }
        
        manifest_path = CONFIG['production_dir'] / 'deployment_manifest.json'
        with open(manifest_path, 'w') as f:
            json.dump(manifest, f, indent=2)
        
        print("✅ Production deployment successful!")
        print(f"   Path: {production_path}")
        print(f"   Version: {timestamp}")
        print(f"   Blue-Green: Enabled with rollback capability")
        print(f"   Monitoring: Active with drift detection")
        
        return {
            'production_path': production_path,
            'metadata': production_metadata,
            'manifest': manifest,
            'deployment_time': datetime.now().isoformat()
        }
        
    except Exception as e:
        print(f"💥 Production deployment failed: {e}")
        return None

# Deploy to production if approved
if staging_deployment and promotion_decision and promotion_decision.get('promote_to_production', False):
    production_deployment = deploy_to_production(staging_deployment, promotion_decision)
    
    if production_deployment:
        print("\n🎊 MODEL SUCCESSFULLY PROMOTED TO PRODUCTION!")
        print("\n📋 Next Steps:")
        print("   1. Restart WHIS API to load new production model")
        print("   2. Monitor performance metrics for 24 hours")
        print("   3. Set up automated alerts for model drift")
        print("   4. Plan next model training cycle")
        
        # Show API restart command
        print("\n🔄 API Restart Commands:")
        print("   # Stop current API")
        print("   pkill -f whis_production_api.py")
        print("   # Start with new production model")
        print("   python whis_production_api.py")
else:
    production_deployment = None
    if promotion_decision and not promotion_decision.get('promote_to_production', False):
        print("\n🚫 Production deployment skipped - promotion not approved")
        print("\n📋 Required Actions:")
        for blocker in promotion_decision.get('blockers', []):
            print(f"   • Fix: {blocker}")
        print("\n🔄 Re-run this pipeline after addressing issues")
    else:
        print("⚠️  Skipping production deployment - no promotion decision available")

## 7. Pipeline Summary

In [ ]:
print("📊 MODEL PROMOTION PIPELINE SUMMARY")
print("=" * 45)

# Collect all pipeline results
pipeline_summary = {
    'pipeline_run_id': f"promotion_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
    'timestamp': datetime.now().isoformat(),
    'stages': {
        'model_discovery': {
            'completed': selected_model is not None,
            'selected_model': selected_model['name'] if selected_model else None
        },
        'validation': {
            'completed': validation_results is not None,
            'passed': validation_results.get('passed', False) if validation_results else False
        },
        'staging_deployment': {
            'completed': staging_deployment is not None,
            'path': str(staging_deployment['staging_path']) if staging_deployment else None
        },
        'integration_testing': {
            'completed': integration_results is not None,
            'passed': integration_results.get('passed', False) if integration_results else False
        },
        'promotion_decision': {
            'completed': promotion_decision is not None,
            'approved': promotion_decision.get('promote_to_production', False) if promotion_decision else False
        },
        'production_deployment': {
            'completed': production_deployment is not None,
            'path': str(production_deployment['production_path']) if production_deployment else None
        }
    },
    'overall_success': production_deployment is not None
}

# Display stage results
print("\n🔄 Pipeline Stages:")
for stage_name, stage_info in pipeline_summary['stages'].items():
    status = "✅" if stage_info.get('completed', False) else "❌"
    print(f"   {status} {stage_name.replace('_', ' ').title()}")
    
    if stage_name == 'validation' and stage_info['completed']:
        passed = "✅ PASSED" if stage_info['passed'] else "❌ FAILED"
        print(f"      Performance validation: {passed}")
    
    elif stage_name == 'integration_testing' and stage_info['completed']:
        passed = "✅ PASSED" if stage_info['passed'] else "❌ FAILED"
        print(f"      Integration tests: {passed}")
    
    elif stage_name == 'promotion_decision' and stage_info['completed']:
        approved = "✅ APPROVED" if stage_info['approved'] else "❌ BLOCKED"
        print(f"      Production promotion: {approved}")

# Overall result
if pipeline_summary['overall_success']:
    print("\n🎉 PIPELINE SUCCESS!")
    print(f"   Model successfully promoted to production")
    print(f"   Production path: {production_deployment['production_path']}")
    print(f"   Ready for API integration")
else:
    print("\n⚠️  PIPELINE INCOMPLETE")
    print(f"   Pipeline stopped at failed validation or integration tests")
    print(f"   Review issues and re-run pipeline")

# Performance summary (if available)
if validation_results and validation_results.get('performance_metrics'):
    print("\n📈 Model Performance Summary:")
    for table_type, metrics in validation_results['performance_metrics'].items():
        auc = metrics.get('auc', 0)
        precision = metrics.get('precision', 0)
        recall = metrics.get('recall', 0)
        inference_ms = metrics.get('inference_time_ms', 0)
        
        print(f"   • {table_type}:")
        print(f"     AUC: {auc:.3f}, Precision: {precision:.3f}, Recall: {recall:.3f}")
        print(f"     Inference: {inference_ms:.1f}ms")

# Save complete pipeline report
report_path = CONFIG['staging_dir'] / f"pipeline_summary_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(report_path, 'w') as f:
    # Convert Path objects to strings for JSON serialization
    json_summary = json.loads(json.dumps(pipeline_summary, default=str))
    json.dump(json_summary, f, indent=2)

print(f"\n💾 Complete pipeline report saved: {report_path}")

print("\n🏁 Model Promotion Pipeline Complete!")
print("\n📋 Next Actions:")
if pipeline_summary['overall_success']:
    print("   1. Restart WHIS API with new production model")
    print("   2. Monitor production metrics for 24-48 hours")
    print("   3. Set up automated model drift monitoring")
    print("   4. Schedule regular retraining pipeline")
else:
    print("   1. Review validation and integration test failures")
    print("   2. Improve model or fix identified issues")
    print("   3. Re-run the promotion pipeline")
    print("   4. Consider ensemble methods or different algorithms")